In [ ]:
# all new rna lines in 21Q3 had wrong internal bam (& sizehash cols) assigned on the tracker
# this notebook fixes 

from __future__ import print_function
import pandas as pd
from gsheets import Sheets
import os 
from depmapomics import tracker

SHEETCREDS = '../.credentials.json'
MY_ID = '~/.client_secret.json'
MYSTORAGE_ID = "~/.storage.json"
SHEETNAME = 'ccle sample tracker'
REFSHEET_URL = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"

In [ ]:
# investigate why internal_bam and internal_bai are swapped within a set of samples
track = Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
track = track.dropna(subset=['internal_bam_filepath', 'internal_bai_filepath'])
track['cds_id'] = track.index

In [ ]:
swappedbams = track[track.apply(lambda x: 'CDS-' in x.internal_bam_filepath and x.cds_id not in x.internal_bam_filepath, axis=1)]

In [ ]:
swappedbams.internal_bam_filepath

In [ ]:
swappedbams.cds_id.tolist()

In [ ]:
# making sure there's no bams missing/broken, just swapped within this set
set(swappedbams.index) == set([x.split('/')[-1][:10] for x in swappedbams['internal_bam_filepath']])

In [ ]:
set(swappedbams.index) == set([x.split('/')[-1][:10] for x in swappedbams['internal_bai_filepath']])

In [ ]:
# pull tracker again, correct internal_bam_filepath, internal_bai_filepath, size, crc32c_hash and md5_hash
track = tracker.getTracker()
for idx in swappedbams.index:
    correct_idx = swappedbams.loc[idx, 'internal_bam_filepath'].split('/')[-1][:10]
    track.loc[correct_idx, 'internal_bam_filepath'] = swappedbams.loc[idx, 'internal_bam_filepath']
    track.loc[correct_idx, 'internal_bai_filepath'] = swappedbams.loc[idx, 'internal_bai_filepath']
    track.loc[correct_idx, 'size'] = swappedbams.loc[idx, 'size']
    track.loc[correct_idx, 'crc32c_hash'] = swappedbams.loc[idx, 'crc32c_hash']
    track.loc[correct_idx, 'md5_hash'] = swappedbams.loc[idx, 'md5_hash']

In [ ]:
sheetname='ccle sample tracker'
sheetcreds='../../.credentials.json'

In [ ]:
from genepy.google.google_sheet import dfToSheet
dfToSheet(track, sheetname, secret=sheetcreds)

In [ ]:
# Update FP terra workspace

In [ ]:
workspace = 'broad-firecloud-ccle/CCLE_SNP_QC'

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

v = tc.get(name='ccle-bam-fingerprints-6f30', version=4, file='fingerprint_lod_matrix')

In [ ]:
# Generate matrix with LOD score for new fingerprint vcfs
new_lod_list = []
samples_df = wm.get_entities("sample_group")['cross_checks_out'].tolist()
for batch in samples_df:
    # could be pd concat
    df = pd.read_csv(batch, sep='\t', comment='#')
    lod_mat = df.pivot(index = "LEFT_SAMPLE",columns="RIGHT_SAMPLE",values = "LOD_SCORE")
    new_lod_list.append(lod_mat)
new_lod_mat = pd.concat(new_lod_list)
new_lod_mat.index.name = None
new_lod_mat = new_lod_mat.T

In [ ]:
df1 = pd.read_csv('gs://fc-secure-6b6a3e1a-6fb8-4d30-b0df-a359e6c5d6e6/d93e690f-d4cc-4190-b62a-4f5f0869f99d/crosscheck/779d4831-255c-4bf1-82c4-905e5cc13e19/call-run_crosscheck/cacheCopy/vcf_batch_6_crosscheck', sep='\t', comment='#')
dfin = df1[df1.LEFT_SAMPLE.isin(swappedidx)]
dfin['LEFT_SAMPLE'].unique()

In [ ]:
dfin[(dfin.RIGHT_SAMPLE == 'CDS-y7yI24') & (dfin.LEFT_SAMPLE == 'CDS-7fm9Do')]

In [ ]:
prev_lod_mat =  tc.get(name='ccle-bam-fingerprints-6f30',file='fingerprint_lod_matrix')
new_ids = set(new_lod_mat.index)
old_ids = set(prev_lod_mat.index) - set(new_ids)
updated_lod_mat = pd.concat((prev_lod_mat.loc[old_ids,old_ids],new_lod_mat.loc[new_ids,old_ids]), axis=0)
updated_lod_mat = pd.concat((updated_lod_mat.loc[new_ids.union(old_ids), old_ids], new_lod_mat.transpose().loc[new_ids.union(old_ids, new_ids)]), axis=1)
updated_lod_mat

In [ ]:
new_lod_mat

In [ ]:
from depmapomics import tracker

ref = Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
ref = ref.loc[ref.blacklist != 1]

In [ ]:
import numpy as np

print("issues with ")
previ = ''
l = {}
for i,j in [(v.index[x], v.columns[y]) for x, y in np.argwhere(v.values>500)]:
    if (i in ref.index and j in ref.index):
        if i == j:
            continue
        if ref.loc[i]['participant_id'] == ref.loc[j]['participant_id']:
            continue
        if i != previ:
            if previ!='':
                l.update({'_'.join(ref.loc[previ, ['arxspan_id','version','datatype','participant_id','stripped_cell_line_name']].astype(str).values.tolist()):n})
            n = [tuple(ref.loc[j, ['arxspan_id','version','datatype','participant_id','stripped_cell_line_name']].values)]
        else:
            n.append(tuple(ref.loc[j, ['arxspan_id','version','datatype','participant_id','stripped_cell_line_name']].values))
        previ = i

In [ ]:
len(l)

In [ ]:
updated_lod_mat.loc['CDS-WfPJyi', 'CDS-WSxpHG']

In [ ]:
updated_lod_mat.loc['CDS-WfPJyi', 'CDS-7fm9Do'] # swapped

In [ ]:
updated_lod_mat.loc['CDS-WfPJyi', 'CDS-9HXF2k'] # 'corrected'

In [ ]:
updated_lod_mat.loc['CDS-y7yI24', 'CDS-9HXF2k']

In [ ]:
previ = ''
l = {}
#ref = ref.append(fbams)
for u in set(fbams.arxspan_id):
    res = v.loc[fbams[fbams.arxspan_id==u].index , ref[ref.arxspan_id==u].index.tolist()]
    for i,j in [(res.index[x], res.columns[y]) for x, y in np.argwhere(res.values<100)]:
        print('__________________________')
        print(res.loc[i,j])
        print(i,':', tuple(ref.loc[i,['arxspan_id','version','datatype','participant_id']].values),j,':',tuple(ref.loc[j,['arxspan_id','version','datatype','participant_id', 'blacklist']]))

In [ ]:
# subset lines in FP workspace that has wrong internal bams
import dalmatian as dm

wm = dm.WorkspaceManager('broad-firecloud-ccle/CCLE_SNP_QC').disable_hound()
sam = wm.get_samples()

In [ ]:
sam.internal_bam_filepath

In [ ]:
track = Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
rnatrack = track[track.datatype == 'rna']
rnatrack

In [ ]:
swappedidx = []
swappedbams = []
vcfs = []
d = {}
bamd = {}
baid = {}
for i in rnatrack.index:
    if i in sam.index and str(sam.loc[i, 'internal_bam_filepath']) != 'nan' and rnatrack.loc[i, 'internal_bam_filepath'] != sam.loc[i, 'internal_bam_filepath']:
        swappedidx.append(i)
        bamd[i] = rnatrack.loc[i, 'internal_bam_filepath']
        baid[i] = rnatrack.loc[i, 'internal_bai_filepath']
        swappedbams.append(sam.loc[i, 'internal_bam_filepath'])
        vcfs.append(sam.loc[i, 'fingerprints'])
        correct_vcf = 'gs://fc-secure-6b6a3e1a-6fb8-4d30-b0df-a359e6c5d6e6/fingerprints/' + sam.loc[i, 'internal_bam_filepath'].split('/')[4][:10] + '.vcf'
        d[sam.loc[i, 'fingerprints']] = correct_vcf
len(swappedidx)

In [ ]:
set(swappedidx) == set(swappedbams)

In [ ]:
len(d)

In [ ]:
for b in swappedbams:
    oldbam = b+'_'
    !gsutil -u broad-firecloud-ccle mv {oldbam} {b}

In [ ]:
for k, v in d.items():
    newk = k + '_'
    !gsutil -u broad-firecloud-ccle mv {k} {newk}

In [ ]:
for k, v in d.items():
    newk = k + '_'
    !gsutil -u broad-firecloud-ccle mv {newk} {v}

In [ ]:
# undo above
for k, v in d.items():
    newv = v + '_'
    !gsutil -u broad-firecloud-ccle mv {newv} {k}

In [ ]:
d

In [ ]:
# now fix the columns in FP workspace
wm = dm.WorkspaceManager('broad-firecloud-ccle/CCLE_SNP_QC').disable_hound()
sam = wm.get_samples()


In [ ]:
bamd

In [ ]:
for k, v in bamd.items():
    sam.loc[k, 'internal_bam_filepath'] = v
    sam.loc[k, 'internal_bai_filepath'] = baid[k]

In [ ]:
sam.loc['CDS-HzoODG', 'internal_bam_filepath']

In [ ]:
sam.loc['CDS-HzoODG', 'internal_bai_filepath']

In [ ]:
wm1 = dm.WorkspaceManager('broad-firecloud-ccle/CCLE_SNP_QC-copy').disable_hound()

In [ ]:
wm1.update_sample_attributes(sam[sam.index.isin(swappedidx)])

In [ ]:
wm1.update_sample_set(sample_set_id='rna_swapped',
                          sample_ids=swappedidx)

In [ ]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/CCLE_SNP_QC').disable_hound()
wm.update_sample_set(sample_set_id='rna_swapped',
                          sample_ids=swappedidx)

In [ ]:
len(swappedidx)

In [ ]:
track = Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
arx = track[track.index.isin(swappedidx)]
arx = set(arx.arxspan_id)
len(arx)

In [ ]:
wgscontaminated = track[(track.arxspan_id.isin(arx)) & (track.datatype == 'wgs')]
wgscontaminated

In [ ]:
#####################

# check lod scores

#####################

In [ ]:
lod_mat = tc.get(name='ccle-bam-fingerprints-6f30',file='fingerprint_lod_matrix')

In [ ]:
prev_id = 'CDS-7fm9Do'
bam_from = 'CDS-WSxpHG'
bam_mapped_to = 'CDS-9HXF2k'
bam_from_wgs = 'CDS-wmOGjh'

In [ ]:
print('wes - rna: ', lod_mat.loc['CDS-WfPJyi', prev_id])
print('wgs - rna: ', lod_mat.loc['CDS-y7yI24', prev_id])
print('wes - wgs: ', lod_mat.loc['CDS-WfPJyi', 'CDS-y7yI24'])

In [ ]:
print('wes - rna: ', lod_mat.loc['CDS-WfPJyi', bam_from])
print('wgs - rna: ', lod_mat.loc['CDS-y7yI24', bam_from])

In [ ]:
print('wes - rna: ', lod_mat.loc['CDS-WfPJyi', bam_mapped_to])
print('wgs - rna: ', lod_mat.loc['CDS-y7yI24', bam_mapped_to])

In [ ]:
lod_mat.loc[bam_from_wgs, bam_from]

In [ ]:
prev_id = 'CDS-f4JXdG'
bam_from = 'CDS-DpgIZ9'
bam_mapped_to = 'CDS-DpgIZ9'

In [ ]:
print('wes - rna: ', lod_mat.loc['CDS-FXX26z', prev_id])
print('wgs - rna: ', lod_mat.loc['CDS-eKPJ34', prev_id])
print('wes - wgs: ', lod_mat.loc['CDS-FXX26z', 'CDS-eKPJ34'])

In [ ]:
print('wes - rna: ', lod_mat.loc['CDS-FXX26z', bam_from])
print('wgs - rna: ', lod_mat.loc['CDS-eKPJ34', bam_from])

In [ ]:
print('wes - rna: ', lod_mat.loc['CDS-FXX26z', bam_mapped_to])
print('wgs - rna: ', lod_mat.loc['CDS-eKPJ34', bam_mapped_to])

In [ ]:
lod_mat.loc['CDS-DpgIZ9', 'CDS-1Zvqir']

In [ ]:
idxd = {}
for k, v in d.items():
    idxd[k.split('/')[4][:10]] = v.split('/')[4][:10]
idxd

In [ ]:
for k, v in idxd.items():
    patient = track.loc[v, 'participant_id']
    patient_idx = list(track[track.participant_id == patient].index)
    for s in patient_idx:
        if k in lod_mat.index and s in lod_mat.index and lod_mat.loc[k, s] >= 500:
            print(k, s)

In [ ]:
print('wes - rna: ', new_lod_mat.loc['CDS-WfPJyi', prev_id])
print('wgs - rna: ', new_lod_mat.loc['CDS-y7yI24', prev_id])
print('wes - wgs: ', new_lod_mat.loc['CDS-WfPJyi', 'CDS-y7yI24'])

In [ ]:
# check if all swapped vcfs are in batches
new_lod_list = []
vcf_groups = wm.get_entities("sample_group")['vcf_group'].tolist()
combined_vcf_dir = {}
for batch in vcf_groups:
    # could be pd concat
    df = pd.read_csv(batch, sep='\t', header=None, comment='#')
    df = df[0].tolist()
    for i in df:
        combined_vcf_dir[i] = batch.split('/')[4]
len(combined_vcf_dir)


In [ ]:
combined_vcf_dir

In [ ]:
combined_vcf_dir
for k, v in d.items():
    if k not in combined_vcf_dir:
        print(k)

In [ ]:
combined_vcf_dir['gs://fc-secure-6b6a3e1a-6fb8-4d30-b0df-a359e6c5d6e6/fingerprints/CDS-WSxpHG.vcf']